In [23]:
import sys
import numpy as np
import math
import importlib
import random
import os
import seaborn as sns

In [24]:
import database_square as sqdb
import database_spsquare as spsqdb

importlib.reload(sqdb)
importlib.reload(spsqdb)

<module 'database_spsquare' from '/Users/rentaguchi/Desktop/code/python/Github_qubit-less FLT/database_spsquare.py'>

In [50]:
invnames = ["Proposed method","BBHL21-GCD method","KH23-GCD method", "TT23-FLT method"]

In [26]:
#Degree of binary elliptic curve (N)

Ns = [163,233,283,571]

In [27]:
#Kim et al.'s quantum multiplication

#n=163,233,283,571

mult_tof = [992,1441,1784,3813]
mult_depth = [10210,16383,22050,61771]
mult_cnot = [76262,154892,224246,862604]

In [28]:
#qubits

qubits = [
            #n=163,233,283,571
    
            [1142,1632,1982,3998], #Proposed method (fewest registers)
            [1157,1647,1998,4015], #BBHL21-GCD
            [1017,1437,1741,3473], #KH23-GCD
            [1957,3030,3963,8566]  #TT23-FLT
        ]

In [68]:
#BBHL21-GCD

def BBHLGCDResource(pm,squareswitch):
    
    N = Ns[pm]
    
    multtof = mult_tof[pm]
    multdepth = mult_depth[pm]
    multcnot = mult_cnot[pm]
    
    
    tof = 0
    cnot = 0
    depth = 0
    
    logn = math.floor(math.log2(N))
    
    
    for j in range(0,2*N-1):
        s = min([2*N-2-j,N])
        t = min([j+1,N])
        tof += 1 + (s + 1) + (t + 1) + (22*logn + 26) + (s + 1) + (t + 1)
        cnot += (logn + 2) + 2*(s + 1) + 2*(t + 1) + (2*logn + 3) + 1 + 1
        depth += N + (logn + 2) + ((s + 1) + (t + 1) + 2) + (12*(logn + 2) - 6) + 1 + (s + 1) + (t + 1)

    depth += N//2
    
    return tof, cnot, depth
    

In [102]:
#KH23-GCD
#Based on "verify algorithm.py" [available] https://zenodo.org/records/7908587

def KHGCDResource(pm,squareswitch):
    
    
    N = Ns[pm]
    tof = 0
    cnot = 0
    depth = 0
    
    def masksize(a,b):
        return min([a,b-1]) - max([(a-1)//2,0]) + 1

    def bitReverse(num, nbit):
        b = '{:0{width}b}'.format(num, width=nbit)
        return int(b[::-1], 2)


    def leftRotate(m, nbit):            
        b = (m & 1)
        m = (m >> 1) | (b << (nbit-1))
        return m


    def bitMult(a, b):                    
        c = 0
        while a.bit_length() > 0:
            if (a&1)==1: c ^= b
            b <<= 1
            a >>= 1
        return c

    def modular(n, m):
        while n.bit_length() >= m.bit_length():
            n ^= (m << (n.bit_length() - m.bit_length()))
        return n


    def CTOF_with_unary_iteration(mask, q, a, b, ell):   # mask, g[0], f, g
        # Lambda = N - 1 - max(floor((ell-1)/2), 0)
        t = 0                                                           # t <- 0
        for i in range(max(math.floor((ell-1)/2), 0), N):                    # unary iteration
            if mask == i and q == 1 and i <= min(ell, N-1):             # unary iteration only for max(floor((ell-1)/2), 0) <= i <= min(ell, n-1)
                t ^= 1                                                  # t <- TOF(t, unary_iteration_i(mask), q)

            if t == 1 and i != N - 1:                                   # skip the last part
                b ^= a & (1 << (N - i - 1))                             # g[Lambda - i - 1] <- TOF(g[Lambda - i - 1], t, f[Lambda - i - 1])

        t ^= q                                                          # t <- CNOT(t, q)
        assert(t == 0)
        return b

    
    if pm == 0:
        R_0 = (1<<163) | (1<<7) | (1<<6) | (1<<3) | 1
    elif pm == 1:
        R_0 = (1<<233) | (1<<74) | 1
    elif pm == 2:
        R_0 = (1<<283) | (1<<12) | (1<<7) | (1<<5) | 1
    elif pm == 3:
        R_0 = (1<<571) | (1<<10) | (1<<5) | (1<<2) | 1
    
    R_1 = 0
    while R_1 == 0:
        for i in range(N): R_1 ^= (random.randint(0,1)<<i)

    f = bitReverse(R_0, N + 1)
    g = bitReverse(R_1, N)
    v, r = 0, 1
    delta = 0
    mask = 0

    a, b = 0, 0
    for i in range(2*N - 1):

        lLambda = N - 1 - max([(i-1)//2,0])         #lLambda = Lambda
        sLambda = min([i+1,N])                        #sLambda = lambda
        v <<= 1                                     # v <- RIGHTSHIFT(v)
        a ^= (delta >= 0) & ((g&1)==1)              # a <- TOF(sign + 1, g[0], a)
        tof += 1
        depth += 1

        if a :
            f, g = g, f                             # f, g <- CSWAP_a(f[n...0],g[n...0])
            v, r = r, v                             # v, r <- CSWAP_a(v[lambda...0],r[lambda...0])
        tof += (N + 1) + (sLambda + 1)
        cnot += 2*(N + 1) + 2*(sLambda + 1)
        depth += (N + 1) + (sLambda + 1) + 2

        g = CTOF_with_unary_iteration(mask, (g&1), f, g, i)  # g[Lambda...1] <- CTOF_WITH_UNARY_ITERATION_i(mask, g[0], f[Lambda...1], g[Lambda...1])

        tof += N + min([i,N-1]) - 2*max([(i-1)//2,0])-1
        cnot += 2*(min([i,N-1]) - max([(i-1)//2,0])+1)

        if i != 0:
            depth += 5*(masksize(i,N)-1) + 2\
            - 2*math.floor(math.log2(masksize(i,N)))\
            + max([2*(math.floor(math.log2(masksize(i,N))))+1,\
                   max([(N-1-max([(i-1)//2,0])) - (masksize(i,N)-1),0]) + 1])
        else:
            depth += 5*(masksize(i,N)-1) + 2\
            - 2*(0)//1\
            + max([2*(0)+1,\
                   max([(N-1-max([(i-1)//2,0])) - (masksize(i,N)-1),0]) + 1])

        a ^= (delta >=0 )                           # a <- CNOT(sign + 1, a)
        cnot += 1
        depth += 1

        if a or delta == -1: 
            mask += 1              # mask <- CINC_(delta=-1|a=1)(mask)
            
            tof += 4*math.floor(math.log2(N)) + 4
            cnot += math.floor(math.log2(N)) + 1
            depth += 5*math.floor(math.log2(N)) + 5
            

        a ^= (delta >= 0)                           # a <- CNOT(sign + 1, a)
        cnot += 1
        depth += 1

        if a: delta = -1 - delta                    # delta <- T_a(delta)
        else: delta = 1 + delta


        tof += 2*math.floor(math.log2(N))+2
        cnot += 2*math.floor(math.log2(N))+4
        depth += 4*math.floor(math.log2(N))+6

        a ^= ((g&1)==1) & ((v&1)==1)                # a <- TOF(v[0], g[0], a)
        depth += 2     

        if (g&1)==1:
            r = r ^ v                               # r <- TOF(g[0], v[lambda...0], r[lambda...0])
        tof += sLambda + 1
        depth += sLambda + 1

        g = leftRotate(g, N+1)                      # g <- LEFTROTATE(g[n...0])

        assert(max(r.bit_length(), v.bit_length()) <= min(i, N) + 1) # lambda
        assert(mask >= max(math.floor((i-1)/2), 0)) # Lambda
        assert(a==0 & b==0)

    inv_R_1 = bitReverse(v, N)
    result = modular(bitMult(inv_R_1, R_1), R_0)
    assert(result == 1)
    return tof, cnot, depth

In [104]:
def ProposedFLTResource(pm,ini,squareswitch):

    
    
    
    #addition chain
    pnPM = [
      [1,2,3,6,9,18,27,54,108,162],
      [1,2,3,4,7,14,28,29,58,116,232],
      [1,2,3,6,9,15,30,45,47,94,141,282],
      [1,2,3,4,7,14,28,29,57,114,171,285,570]
     ]

    #register bounded addition chains
    
    
    anPM = [
      [
          [1,1,3,3,3,1,1,9,9,27,9,9,54,54],
          [1,1,3,3,9,3,1,1,9,27,54,9,54],
          [1,1,3,3,9,9,1,1,27,54,54,9],
          [1,1,3,3,9,9,27,1,54,54,9],
          [1,1,3,3,9,9,27,54,1,54]
      ],
      [
          [1,1,1,3,1,1,1,7,14,1,14,7,29,58,29,116],
          [1,1,1,3,7,1,1,14,1,14,7,29,58,29,116],
          [1,1,1,3,7,14,7,1,1,1,29,58,29,116],
          [1,1,1,3,7,14,1,7,1,1,29,58,116],
          [1,1,1,3,7,14,1,29,7,1,58,116],
          [1,1,1,3,7,14,1,29,58,1,116]
      ],
      [  
          [1,1,3,3,6,3,3,1,15,15,2,15,15,1,47,47,47,141],
          [1,1,3,3,6,3,3,1,15,15,2,15,15,1,47,47,141],
          [1,1,3,3,6,15,15,3,3,1,2,47,15,1,47,141],
          [1,1,3,3,6,15,15,2,15,3,1,1,47,47,141],
          [1,1,3,3,6,15,15,2,15,1,1,47,47,141],
          [1,1,3,3,6,15,15,2,47,15,1,47,141],
          [1,1,3,3,6,15,15,2,47,47,1,141]
      ],
      [
          [1,1,1,3,1,1,1,7,14,1,28,1,14,7,57,57,114,57,57,285],
          [1,1,1,3,7,1,1,1,14,1,28,1,14,7,57,57,114,57,285],
          [1,1,1,3,7,14,1,1,1,1,28,57,57,1,14,7,114,285],
          [1,1,1,3,7,14,1,1,1,1,28,57,57,1,7,114,285],
          [1,1,1,3,7,14,1,28,1,1,1,57,57,114,1,285],
          [1,1,1,3,7,14,1,28,1,1,1,57,57,114,285],
          [1,1,1,3,7,14,1,28,1,1,57,57,114,285],
          [1,1,1,3,7,14,1,28,1,57,57,114,285]
      ]
     ]

    bnPM = [
      [
          [1,2,3,6,3,2,1,9,18,27,18,9,54,108],
          [1,2,3,6,9,3,2,1,18,27,54,18,108],
          [1,2,3,6,9,18,2,1,27,54,108,18],
          [1,2,3,6,9,18,27,2,54,108,18],
          [1,2,3,6,9,18,27,54,2,108]
      ],
      [
          [1,2,3,4,3,2,1,7,14,28,14,7,29,58,29,116],
          [1,2,3,4,7,2,1,14,28,14,7,29,58,29,116],
          [1,2,3,4,7,14,7,2,1,28,29,58,29,116],
          [1,2,3,4,7,14,28,7,2,1,29,58,116],
          [1,2,3,4,7,14,28,29,7,1,58,116],
          [1,2,3,4,7,14,28,29,58,1,116]
      ],
      [
          [1,2,3,6,9,6,3,2,15,30,45,30,15,1,47,94,47,141],
          [1,2,3,6,9,6,3,2,15,30,45,30,15,1,47,94,141],
          [1,2,3,6,9,15,30,6,3,2,45,47,30,1,94,141],
          [1,2,3,6,9,15,30,45,30,3,2,1,47,94,141],
          [1,2,3,6,9,15,30,45,30,2,1,47,94,141],
          [1,2,3,6,9,15,30,45,47,30,2,94,141],
          [1,2,3,6,9,15,30,45,47,94,2,141]
      ],
      [
          [1,2,3,4,3,2,1,7,14,28,29,28,14,7,57,114,171,114,57,285],
          [1,2,3,4,7,3,2,1,14,28,29,28,14,7,57,114,171,114,285],
          [1,2,3,4,7,14,28,3,2,1,29,57,114,28,14,7,171,285],
          [1,2,3,4,7,14,28,3,2,1,29,57,114,28,7,171,285],
          [1,2,3,4,7,14,28,29,3,2,1,57,114,171,28,285],
          [1,2,3,4,7,14,28,29,3,2,1,57,114,171,285],
          [1,2,3,4,7,14,28,29,2,1,57,114,171,285],
          [1,2,3,4,7,14,28,29,1,57,114,171,285]
      ]
     ]

    
    
    N = Ns[pm]
   
    
    multtof = mult_tof[pm]
    multdepth = mult_depth[pm]
    multcnot = mult_cnot[pm]

    pn = pnPM[pm]

    an = anPM[pm][ini]
    bn = bnPM[pm][ini]
      
        
    length = len(pn)
    steps = len(an)
        
    
    sqdepth = sqdb.squareLUP_depth[pm][1]
    sqcnot = sqdb.squareLUP_cnot[pm][1]

    sqedtimes = [0 for i in range(length)]

    tof = 0
    depth = 0
    cnot = 0
    
    
    def index(n):
        for i in range(length):
            if pn[i] == n:
                return i
        return

    for i in range(steps):

        #二倍算の場合
        if an[i] == bn[i]:

            #ADD
            depth += 1
            cnot += N

        #目的の二乗回数と現状の二乗回数の差
        dif = an[i] - sqedtimes[index(bn[i])]
        
        if squareswitch == 0:
            
            #SQUAREを繰り返し適用する場合

            #bnをSQUAREまたはSQUARE-1して目的の二乗回数に
            for j in range(abs(dif)):
                depth += sqdepth
                cnot += sqcnot

            #anをSQUARE-1して二乗回数0に
            for j in range(sqedtimes[index(an[i])]):
                depth += sqdepth
                cnot += sqcnot

            #an+bnをSQUARE-1して二乗回数0に
            for j in range(sqedtimes[index(an[i]+bn[i])]):
                depth += sqdepth
                cnot += sqcnot
            
        elif squareswitch == 1:
            #SQUAREを最適化する場合

            #bnをSQUAREまたはSQUARE-1して目的の二乗回数に
            if an[i] == bn[i]:
                depth += spsqdb.spsquareALLNOPT_depth[pm][abs(dif)] - 1
                cnot += spsqdb.spsquareALLNOPT_cnot[pm][abs(dif)] - N
            else:
                depth += sqdb.squareOPT_depth[pm][abs(dif)]
                cnot += sqdb.squareOPT_cnot[pm][abs(dif)]

            #anをSQUARE-1して二乗回数0に
            depth += sqdb.squareOPT_depth[pm][sqedtimes[index(an[i])]]
            cnot += sqdb.squareOPT_cnot[pm][sqedtimes[index(an[i])]]

            #an+bnをSQUARE-1して二乗回数0に
            depth += sqdb.squareOPT_depth[pm][sqedtimes[index(an[i]+bn[i])]]
            cnot += sqdb.squareOPT_cnot[pm][sqedtimes[index(an[i]+bn[i])]]
        
        else:
            sys.exit()

        #MULT
        depth += multdepth
        cnot += multcnot
        tof += multtof

        #二倍算の場合
        if an[i] == bn[i]:
            
            if squareswitch == 0:
                
                #SQUAREを繰り返し適用する場合

                #bnをSQUARE-1して二乗回数0に
                for j in range(an[i]):
                    depth += sqdepth
                    cnot += sqcnot
                
            elif squareswitch == 1:

                #SQUAREを最適化する場合

                #bnをSQUARE-1して二乗回数0に
                depth += spsqdb.spsquareALLNOPT_depth[pm][an[i]] - 1
                cnot += spsqdb.spsquareALLNOPT_cnot[pm][an[i]] - N

            else:
                sys.exit()
                
            #ADD
            depth += 1
            cnot += N

            #anの二乗回数を変更
            sqedtimes[index(an[i])] = 0

        #加算の場合
        else:

            #anの二乗回数を変更
            sqedtimes[index(an[i])] = 0

            #bnの二乗回数を変更
            sqedtimes[index(bn[i])] = an[i]

        #an+bnの二乗回数を変更
        sqedtimes[index(an[i]+bn[i])] = 0
        
        
    #SQUARE
    
    depth += sqdepth
    cnot += sqcnot

    return tof, cnot, depth

In [86]:
#TT23-FLT

def TTFLTResource(pm,squareswitch):


    #addition chains
    pns = [
        [1,2,3,6,9,18,27,54,108,162],
        [1,2,3,4,7,14,28,29,58,116,232],
        [1,2,3,6,9,15,30,45,47,94,141,282],
        [1,2,3,4,7,14,28,29,57,114,171,285,570]
    ]
    
    
    ans = [
          [0,1,2,3,4,5,6,7,8],
          [0,1,2,2,4,5,6,7,8,9],
          [0,1,2,3,3,5,6,7,8,9,10],
          [0,1,2,2,4,5,6,6,8,9,9,11]
         ]
    bns = [
        [0,0,2,2,4,4,6,7,7],
        [0,0,0,3,4,5,0,7,8,9],
        [0,0,2,2,4,5,5,1,8,8,10],
        [0,0,0,3,4,5,0,7,8,8,10,11]
    ]
    
    Qns = [
          [1,1,3,3,9,9,27,54,54],
          [1,1,1,3,7,14,1,29,58,116],
          [1,1,3,3,6,15,15,2,47,47,94],
          [1,1,1,3,7,14,1,28,29,57,114,285]
         ]
    
    cls = [
        [0,-1,-3,-9,-27,-54],
        [0,-1,-7,-14,-29,-58,-116],
        [0,-1,-3,-15,-47,-141],
        [0,-1,-7,-14,-57,-285]
    ]
    
    pn = pns[pm]
    an = ans[pm]
    bn = bns[pm]
    Qn = Qns[pm]
    cl = cls[pm]
    
    
    N = Ns[pm]
    L = len(an)

    
    multtof = mult_tof[pm]
    multdepth = mult_depth[pm]
    multcnot = mult_cnot[pm]
    
        
    sqdepth = sqdb.squareLUP_depth[pm][1]
    sqcnot = sqdb.squareLUP_cnot[pm][1]

    tof = 0
    depth = 0
    cnot = 0



    for i in range(L):
        if an[i] == bn[i]:

            depth += 1
            cnot += N
            if squareswitch == 0:
                for j in range(Qn[i]):
                    depth += sqdepth
                    cnot += sqcnot
            elif squareswitch == 1:
                depth += spsqdb.spsquareALLNOPT_depth[pm][Qn[i]] - 1
                cnot += spsqdb.spsquareALLNOPT_cnot[pm][Qn[i]] - N
            else:
                sys.exit()

            depth += multdepth
            cnot += multcnot
            tof += multtof


        else:
            if squareswitch == 0:
                for j in range(Qn[i]):  
                    depth += sqdepth
                    cnot += sqcnot
            elif squareswitch == 1:
                depth += sqdb.squareOPT_depth[pm][Qn[i]]
                cnot += sqdb.squareOPT_cnot[pm][Qn[i]]
            else:
                sys.exit()
                
            depth += multdepth
            cnot += multcnot
            tof += multtof

    for i in range(len(cl)):
        if squareswitch == 0:    
            depth += abs(cl[i])*sqdepth
            cnot += abs(cl[i])*sqcnot
        elif squareswitch == 1:
            depth += spsqdb.spsquareALLNOPT_depth[pm][abs(cl[i])] - 1
            cnot += spsqdb.spsquareALLNOPT_cnot[pm][abs(cl[i])] - N
        else:
            sys.exit()
            
        depth += 1
        cnot += N
    
    depth += sqdepth
    cnot += sqcnot
    
    
    return tof, cnot, depth



In [96]:
#Shor's algorithm resource estimate by previous method

def ShorResource(pm,tof,cnot,depth,invswitch):
    

    lmax = 20
    minregister = 5
    N = Ns[pm]  
    
    multtof = mult_tof[pm]
    multdepth = mult_depth[pm]
    multcnot = mult_cnot[pm]    
    
    spsqcnot = spsqdb.spsquareALLNOPT_cnot[pm][1]
    spsqdepth = spsqdb.spsquareALLNOPT_depth[pm][1]
    
    
    #single point addition---------------------------------------------------------------------------
    
    totaldepth = 1 + N//2 + depth + multdepth + multdepth + depth + spsqdepth + N//2 + N \
                    + N + spsqdepth + depth + multdepth + multdepth + depth + 1 + N//2 + N
    totaltof = 0 + 0 + tof + multtof + multtof + tof + 0 + 0 + N + N + 0 + tof + multtof \
                    + multtof + tof + 0 + N + 0
    totalcnot = 0 + N//2 + cnot + multcnot + multcnot + cnot + spsqcnot + N//2 + 0 + 0 \
                + spsqcnot + cnot + multcnot + multcnot + cnot + 0 + 0 + N//2
    
    totalqubit = qubits[invswitch][pm]
    
    #--------------------------------------------------------------------------------------------------
    
    
    
    
    #Windowing----------------------------------------------------------------------------------------
    
    mintof = 10**20
    minl = -1
    for l in range(1,lmax):
        
        
        lookuptof = 2*(2**l-1)

        wintotaltof = lookuptof + 0 + 0 + lookuptof + tof + multtof + multtof + tof + 0 + 0 \
                        + lookuptof + 0 + 0 + 0 + lookuptof + 0 + tof + multtof + multtof + tof \
                            + lookuptof + 0 + 0 + 0 + lookuptof
        
        
        
        wintotaltof *= math.ceil((N+1)/l)*2 
        
        wintotaltof += totaltof 
        
        if wintotaltof < mintof:
            mintof = wintotaltof
            minl = l
    
    #--------------------------------------------------------------------------------------------------
    
    
    
    l = 2*N + 2
    print(
        "N:",N,
        "qubits:",format(totalqubit,','),
        "TOF:",format(totaltof*l,','),
        "depth:",format(totaldepth*l,','),
        "CNOT:",format(totalcnot*l,','),
        "window:",minl,
        "windowTOF:",format(mintof,','),
        "QV:",'{:.2e}'.format(totalqubit*totaldepth*l,','),
        "QT:",'{:.2e}'.format(totalqubit*totaltof*l,',')
    )
    
    
    
    return totaltof*l, totalcnot*l, totaldepth*l


In [97]:
#Shor's algorithm resource estimate by proposed method


def ShorResourceQLF(pm,ini,tof,cnot,depth,invswitch):
    
    lmax = 20
    minregister = 5
    N = Ns[pm]  
    
    multtof = mult_tof[pm]
    multdepth = mult_depth[pm]
    multcnot = mult_cnot[pm]
        
    spsqcnot = spsqdb.spsquareALLNOPT_cnot[pm][1]
    spsqdepth = spsqdb.spsquareALLNOPT_depth[pm][1]
    
    
    #single point addition---------------------------------------------------------------------------
    
    totaldepth = 1 + N//2 + depth + multdepth + multdepth + depth + spsqdepth + N//2 \
                    + N + N + spsqdepth + depth + multdepth + multdepth + depth + 1 + N//2 + N
    totaltof = 0 + 0 + tof + multtof + multtof + tof + 0 + 0 + N + N + 0 + tof + multtof \
                + multtof + tof + 0 + N + 0
    totalcnot = 0 + N//2 + cnot + multcnot + multcnot + cnot + spsqcnot + N//2 + 0 + 0 + \
                spsqcnot + cnot + multcnot + multcnot + cnot + 0 + 0 + N//2
    
    totalqubit = qubits[invswitch][pm] + N*ini
    
    #--------------------------------------------------------------------------------------------------
    
    
    #Windowing----------------------------------------------------------------------------------------
    mintof = 10**20
    minl = -1
    for l in range(1,lmax):
        
        
        lookuptof = 2*(2**l-1)

        wintotaltof = lookuptof + 0 + 0 + lookuptof + tof + multtof + multtof + tof \
                        + 0 + 0 + lookuptof + 0 + 0 + 0 + lookuptof + 0 + tof + multtof \
                            + multtof + tof + lookuptof + 0 + 0 + 0 + lookuptof
        
        
        
        wintotaltof *= math.ceil((N+1)/l)*2 
        
        wintotaltof += totaltof 
        
        if wintotaltof < mintof:
            mintof = wintotaltof
            minl = l
    #--------------------------------------------------------------------------------------------------
    
    l = 2*N + 2
    print(
          "N:",N,
          "registers:",ini+minregister,
          "qubits:",format(totalqubit,','),
          "TOF:",format(totaltof*l,','),
          "depth:",format(totaldepth*l,','),
          "CNOT:",format(totalcnot*l,','),
          "window:",minl,\
          "windowTOF:",format(mintof,','),
          "QV:",'{:.2e}'.format(totalqubit*totaldepth*l,','),
          "QT:",'{:.2e}'.format(totalqubit*totaltof*l,',')
         )
    return totaltof*l, totalcnot*l, totaldepth*l

In [101]:
def ResourceEstimate(squareswitch, invswitch,pmlist):
    
    func_dict = {
        '0' : ProposedFLTResource,
        '1' : BBHLGCDResource,
        '2' : KHGCDResource,
        '3' : TTFLTResource
    }
    
    
    numberofchains = [5,6,7,8]
    
    inv = str(invswitch)   
    
    INVNAMES = []   

    for i in INVLIST:
        INVNAMES.append(invnames[i])
    
    
    
    if invswitch == 0:  #Proposed method
        
        tofss = []
        cnotss = []
        depthss = []
        
        for pm in pmlist: 
            chains = numberofchains[pm]
            
            tofs = [0 for i in range(chains)]
            cnots = [0 for i in range(chains)]
            depths = [0 for i in range(chains)]

            for ini in range(chains):

                tofs[ini], cnots[ini], depths[ini] = func_dict[inv](pm, ini, squareswitch)
                tofs[ini], cnots[ini], depths[ini] \
                = ShorResourceQLF(pm,ini,tofs[ini], cnots[ini], depths[ini],invswitch)

            tofss.append(tofs)
            cnotss.append(cnots)
            depthss.append(depths)
            
        return tofss, cnotss, depthss
        
        
    else:  #Previous method
        
        tofs = [0 for i in range(len(pmlist))]
        cnots = [0 for i in range(len(pmlist))]
        depths = [0 for i in range(len(pmlist))]
        count = 0
        
        for pm in pmlist: 

            tofs[count], cnots[count], depths[count] = func_dict[inv](pm, squareswitch)
            tofs[count], cnots[count], depths[count] \
            = ShorResource(pm,tofs[count], cnots[count], depths[count],invswitch)
            count += 1
        return tofs, cnots, depths
        

In [107]:
#MAIN

"------------------------------------------------------------------------------------------------------------"


SQUARESWITCHLIST = [1,1,1,1]    #0:previous squaring, 1:proposed squaring

INVLIST = [0,1,2,3]             #0:Proposed method, 1:BBHL21-GCD method, 2:KH23-GCD method, 3:TT23-FLT method 
        
PMLIST = [0]                    #0:n=163, 1:n=233, 2:n=283, 3:n=571



"------------------------------------------------------------------------------------------------------------"


INVNAMES = []   

for i in INVLIST:
    INVNAMES.append(invnames[i])

tofs = [[] for i in range(len(INVLIST))]
cnots = [[] for i in range(len(INVLIST))]
depths = [[] for i in range(len(INVLIST))]

count = 0

for j in range(len(INVLIST)):
    i = INVLIST[j]
    SQUARESWITCH = SQUARESWITCHLIST[j]
    print("[",invnames[i],"]")
    tofs[count], cnots[count], depths[count] = ResourceEstimate(SQUARESWITCH, i, PMLIST)
    count += 1
    print('')


[ Proposed method ]
N: 163 registers: 5 qubits: 1,142 TOF: 19,682,952 depth: 233,563,552 CNOT: 1,672,852,808 window: 10 windowTOF: 2,501,073 QV: 2.67e+11 QT: 2.25e+10
N: 163 registers: 6 qubits: 1,305 TOF: 18,381,448 depth: 215,843,680 CNOT: 1,537,254,984 window: 10 windowTOF: 2,362,193 QV: 2.82e+11 QT: 2.40e+10
N: 163 registers: 7 qubits: 1,468 TOF: 17,079,944 depth: 200,721,568 CNOT: 1,431,709,832 window: 10 windowTOF: 2,223,313 QV: 2.95e+11 QT: 2.51e+10
N: 163 registers: 8 qubits: 1,631 TOF: 15,778,440 depth: 186,465,376 CNOT: 1,330,132,168 window: 10 windowTOF: 2,084,433 QV: 3.04e+11 QT: 2.57e+10
N: 163 registers: 9 qubits: 1,794 TOF: 14,476,936 depth: 173,069,856 CNOT: 1,230,076,424 window: 9 windowTOF: 1,935,777 QV: 3.10e+11 QT: 2.60e+10

[ BBHL21-GCD method ]
N: 163 qubits: 1,157 TOF: 288,641,640 depth: 341,963,616 CNOT: 322,348,232 window: 13 windowTOF: 26,303,013 QV: 3.96e+11 QT: 3.34e+11

[ KH23-GCD method ]
N: 163 qubits: 1,017 TOF: 243,048,328 depth: 319,284,384 CNOT: 391,6